In [1]:
"""~Wherobots setup"""

from sedona.spark import *
from sedona.spark import SedonaContext, Adapter
from contextlib import contextmanager
import time
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import desc
from sedona.core.formatMapper.shapefileParser import ShapefileReader

config = SedonaContext.builder().appName('sedona-test').getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
""""""

print(f'master: {sedona.conf.get("spark.master")}')
num_partitions = sc.defaultParallelism
print(f"sc.defaultParallelism: {num_partitions}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/14 15:42:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


master: local[*]
sc.defaultParallelism: 10


In [4]:

# from sedona.spark import *
# from sedona.spark import SedonaContext, Adapter
# from contextlib import contextmanager
# import time
# from pyspark.sql.functions import col, expr
# from pyspark.sql.functions import desc
# from sedona.core.formatMapper.shapefileParser import ShapefileReader

# config = SedonaContext.builder().appName('sedona-test').getOrCreate()

# sedona = SedonaContext.create(config)
# sc = sedona.sparkContext
# print(f'master: {sedona.conf.get("spark.master")}')
# print(f"sc.defaultParallelism: {sc.defaultParallelism}")

# """"""
# num_partitions = sc.defaultParallelism
# print(f"using partitions: {num_partitions}")
from pyspark.sql.functions import regexp_replace
from sedona.sql.types import GeometryType
from sedona.core.enums.grid_type import GridType
from sedona.core.enums.index_type import IndexType
# from geospark.utils.adapter import Adapter


total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")
    
def chudu(df):
    
    df = df.toPandas()
    rows, cols = df.shape
    print(f"Total rows: {rows}, Total columns: {cols}")
    
    memory = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage: {memory:.2f} MB")
    
    print("\nFirst 5 rows:")
    print(df.head(5))
    
    unique_fields = [col for col in df.columns if df[col].is_unique]
    print("\nCols with unique values no repeats:")
    print(unique_fields if unique_fields else "None")
    
    nan_columns = df.isna().sum()
    nan_columns = nan_columns[nan_columns > 0]
    print("\nColumns with NaN:")
    print(nan_columns if not nan_columns.empty else "None")
    print()
    print()

# countries_path = "./my_data/data_EU/countries_shp/"
# countries_rdd = ShapefileReader.readToGeometryRDD(sc, countries_path)

italy_rdd = ShapefileReader.readToGeometryRDD(sc, "./my_data/data_Italy/merged/merged_pop_geom/")
italy_rdd.analyze()
italy_rdd.spatialPartitioning(GridType.KDBTREE)
italy_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)
italy_df = Adapter.toDf(italy_rdd, sedona)

lomb_rdd = ShapefileReader.readToGeometryRDD(sc, "./my_data/data_Italy/lomb/")
lomb_rdd.analyze()
lomb_rdd.spatialPartitioning(GridType.KDBTREE)
lomb_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)
lomb_df = Adapter.toDf(lomb_rdd, sedona)




# grids_df = sedona.read.parquet("./my_data/data_EU/census_grid_EU/grids.parquet")
# grids_df = grids_df.withColumn("geom", expr("ST_GeomFromWKB(geom)"))
# grids_df = grids_df.withColumn(
#     "GRD_ID", regexp_replace("GRD_ID", "CRS3035RES1000m", "")
# )
# grids_df = grids_df.select("GRD_ID", "T", "geom")
# grids_rdd = Adapter.toSpatialRdd(grids_df)
# grids_rdd.analyze()
# grids_rdd.boundaryEnvelope

# countries_rdd.analyze()

# countries_rdd.spatialPartitioning(GridType.KDBTREE)
# countries_rdd.spatialPartitioning(com_rdd.getPartitioner())
# grids_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)
# countries_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)

# countries_df = Adapter.toDf(countries_rdd, sedona)
# countries_df = countries_df.select("CNTR_ID", "NAME_ENGL", "geometry")


# grids_df = grids_df.withColumn("geom", expr("ST_AsText(geom)"))

italy_df = italy_df.repartition(num_partitions * 2)
lomb_df = lomb_df.repartition(num_partitions * 2)

joined_df = italy_df.alias("it").join(
    lomb_df.alias("itt"),
    expr("ST_Intersects(it.geometry, itt.geometry)"),
    "inner"
)
# chudu(joined_df)

# print(joined_df.count())

In [5]:
italy_df = italy_df.repartition(num_partitions * 2)
lomb_df = lomb_df.repartition(num_partitions * 2)

joined_df = italy_df.alias("it").join(
    lomb_df.alias("itt"),
    expr("ST_Intersects(it.geometry, itt.geometry)"),
    "inner"
)

joined_df.count()


25/01/14 14:47:09 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.
Exception in thread "Spark Context Cleaner" java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.util.Utils$.tryOrStopSparkContext(Utils.scala:1477)
	at org.apache.spark.ContextCleaner.org$apache$spark$ContextCleaner$$keepCleaning(ContextCleaner.scala:189)
	at org.apache.spark.ContextCleaner$$anon$1.run(ContextCleaner.scala:79)
25/01/14 14:47:23 ERROR Executor: Exception in task 21.0 in stage 27.0 (TID 119)
java.lang.OutOfMemoryError: Java heap space
	at org.apache.sedona.common.geometrySerde.GeometrySerde.readGeometry(GeometrySerde.java:138)
	at org.apache.sedona.common.geometrySerde.GeometrySerde.read(GeometrySerde.java:97)
	at com.esotericsoftware.kryo.Kryo.readClassAndObject(Kryo.java:813)
	at org.apache.spark.serializer.KryoDeserializationStream.readObject(KryoSerializer.scala:311)
	at org.apache.spark.serializer.DeserializationStream.readValue(Serializer.scala:158)
	a

ConnectionRefusedError: [Errno 111] Connection refused

In [6]:
italy_df.count()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
# from pyspark.sql.functions import regexp_replace
# from sedona.sql.types import GeometryType


from sedona.spark import *
from sedona.spark import SedonaContext, Adapter
from contextlib import contextmanager
import time
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import desc
from sedona.core.formatMapper.shapefileParser import ShapefileReader

from pyspark.sql.functions import regexp_replace
from sedona.sql.types import GeometryType

config = SedonaContext.builder().appName('sedona-test').getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
print(f'master: {sedona.conf.get("spark.master")}')
print(f"sc.defaultParallelism: {sc.defaultParallelism}")

""""""
num_partitions = sc.defaultParallelism
print(f"using partitions: {num_partitions}")


total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")
    
def chudu(df):
    
    df = df.toPandas()
    rows, cols = df.shape
    print(f"Total rows: {rows}, Total columns: {cols}")
    
    memory = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage: {memory:.2f} MB")
    
    print("\nFirst 5 rows:")
    print(df.head(5))
    
    unique_fields = [col for col in df.columns if df[col].is_unique]
    print("\nCols with unique values no repeats:")
    print(unique_fields if unique_fields else "None")
    
    nan_columns = df.isna().sum()
    nan_columns = nan_columns[nan_columns > 0]
    print("\nColumns with NaN:")
    print(nan_columns if not nan_columns.empty else "None")
    print()
    print()

countries_path = "./my_data/data_EU/countries_shp/"
countries_rdd = ShapefileReader.readToGeometryRDD(sc, countries_path)
countries_df = Adapter.toDf(countries_rdd, sedona)

grids_df = sedona.read.parquet("./my_data/data_EU/census_grid_EU/grids.parquet")
# grids_df = grids_df.withColumn("geom", grids_df["geom"].cast(GeometryType))
grids_df = grids_df.withColumn("geom", expr("ST_GeomFromWKB(geom)"))

# grids_df = grids_df.withColumn("geom", expr("ST_AsText(geom)"))

grids_df = grids_df.withColumn(
    "GRD_ID", regexp_replace("GRD_ID", "CRS3035RES1000m", "")
)
grids_df = grids_df.select("GRD_ID", "T", "geom")
countries_df = countries_df.select("CNTR_ID", "NAME_ENGL", "geometry")

grids_df = grids_df.repartition(16)

joined_df = grids_df.alias("grd").join(
    countries_df.alias("con"),
    expr("ST_Intersects(grd.geom, con.geometry)"),
    "inner"
)
# chudu(joined_df)

print(joined_df.count())

In [ ]:
grids_df.printSchema()
countries_df.printSchema()


In [1]:
""" SPATIAL PARTITIOING. DF JOIN """

from sedona.spark import *
from sedona.spark import SedonaContext, Adapter
from contextlib import contextmanager
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import desc
import time
from sedona.core.formatMapper.shapefileParser import ShapefileReader

config = SedonaContext.builder().appName('sedona-example-python')\
    .config('spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider','org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')\
    .getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
print(f'master: {sedona.conf.get("spark.master")}')
print(f"def par: {sc.defaultParallelism}")

""""""
# num_partitions = sc.defaultParallelism * 2
num_partitions = 16
print(f"using partitions: {num_partitions}")

print()
print("DF JOIN")
print()

total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")

cens_shp_path = "/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/"
com_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/"

with get_time("Reading cens shp file"):
    cens_rdd = ShapefileReader.readToGeometryRDD(sc, cens_shp_path)

with get_time("Reading com shp file"):
    com_rdd = ShapefileReader.readToGeometryRDD(sc, com_shp_path)
    

with get_time("Spatial Partitioning"):
    cens_rdd.analyze()
    com_rdd.analyze()

    cens_rdd.spatialPartitioning(GridType.KDBTREE)
    com_rdd.spatialPartitioning(GridType.KDBTREE)
    
cens_df = Adapter.toDf(cens_rdd, sedona)
com_df = Adapter.toDf(com_rdd, sedona)
com_df_renamed = com_df.selectExpr(*[f"{c} AS {c}_com" for c in com_df.columns])

joined_df = (
    com_df_renamed.select("COMUNE_com", "geometry_com")
    .alias("com")
    .join(
        cens_df.alias("cens"),
        expr("ST_Contains(com.geometry_com, cens.geometry)"),
        "inner"
    )
)

with get_time("toDf + rename cols + Spatial Join"):
    joined_df.write.format('geoparquet').mode('overwrite').save('./res_geoparquet')
    # print(f"Total features after join: {joined_df.count()}")

if((total_time)/60 < 1):
    print(f"Job done in {total_time:.2f} sec")
else:
    print(f"Job done in {(total_time)/60:.2f} min")
print()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/07 11:28:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


master: local[*]
def par: 8
using partitions: 16

DF JOIN

Reading cens shp file... DONE in 0.52 sec
Reading com shp file... DONE in 0.31 sec


Spatial Partitioning... DONE in 5.68 sec


24/12/07 11:28:51 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


toDf + rename cols + Spatial Join... DONE in 50.42 sec
Job done in 56.93 sec



In [1]:
""" repartition(8, "column") DF JOIN """

from sedona.spark import *
from sedona.spark import SedonaContext, Adapter
from contextlib import contextmanager
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import desc
import time
from sedona.core.formatMapper.shapefileParser import ShapefileReader

config = SedonaContext.builder().appName('sedona-example-python')\
    .config('spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider','org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')\
    .getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
print(f'master: {sedona.conf.get("spark.master")}')
print(f"def par: {sc.defaultParallelism}")

""""""
# num_partitions = sc.defaultParallelism * 2
num_partitions = 8
print(f"using partitions: {num_partitions}")

print()
print("DF JOIN")
print()

total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")

cens_shp_path = "/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/"
com_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/"

with get_time("Reading cens shp file"):
    cens_rdd = ShapefileReader.readToGeometryRDD(sc, cens_shp_path)

with get_time("Reading com shp file"):
    com_rdd = ShapefileReader.readToGeometryRDD(sc, com_shp_path)
    
cens_df = Adapter.toDf(cens_rdd, sedona).repartition(num_partitions, "COD_UTS")
com_df = Adapter.toDf(com_rdd, sedona)
com_df_renamed = com_df.selectExpr(*[f"{c} AS {c}_com" for c in com_df.columns]).repartition(num_partitions, "COD_UTS_com")

joined_df = (
    com_df_renamed.select("COMUNE_com", "geometry_com")
    .alias("com")
    .join(
        cens_df.alias("cens"),
        expr("ST_Contains(com.geometry_com, cens.geometry)"),
        "inner"
    )
)

with get_time("toDf + rename cols + Spatial Join"):
    joined_df.write.format('geoparquet').mode('overwrite').save('./res_geoparquet')
    # print(f"Total features after join: {joined_df.count()}")

if((total_time)/60 < 1):
    print(f"Job done in {total_time:.2f} sec")
else:
    print(f"Job done in {(total_time)/60:.2f} min")
print()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/07 11:42:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


master: local[*]
def par: 8
using partitions: 8

DF JOIN

Reading cens shp file... DONE in 0.47 sec
Reading com shp file... DONE in 0.29 sec


24/12/07 11:42:47 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


toDf + rename cols + Spatial Join... DONE in 29.21 sec
Job done in 29.98 sec



In [8]:
cens_df.show()

+--------------------+-------+-------+-------+-------+------------+----------+--------+----------+-------+--------+---------+---------+----------+----------+--------+--------+--------+-----+-----+-----------------+-----------------+
|            geometry|COD_REG|COD_UTS|PRO_COM|  SEZ21|    SEZ21_ID|COD_TIPO_S|TIPO_LOC|  LOC21_ID|COD_ZIC|COD_ISAM|COD_ACQUE|COD_ISOLE|COD_MONT_D|COD_AREA_S|COM_ASC1|COM_ASC2|COM_ASC3|POP21|FAM21|       SHAPE_Leng|       SHAPE_Area|
+--------------------+-------+-------+-------+-------+------------+----------+--------+----------+-------+--------+---------+---------+----------+----------+--------+--------+--------+-----+-----+-----------------+-----------------+
|POLYGON ((516786....|      3|    215|  15146|   1528|151460001528|         1|       1|1514610006|      0|       0|        0|        0|         0|         0|15146003|15146021|       0|  197|  120|4.58618956384e+02|8.24104927375e+03|
|POLYGON ((539453....|      3|     98|  98031|     13|980310000013| 

In [9]:
com_df.show()

+--------------------+-------+-------+--------+------+-------+-------+---------+-------------------+--------+------+------------------+------------------+
|            geometry|COD_RIP|COD_REG|COD_PROV|COD_CM|COD_UTS|PRO_COM|PRO_COM_T|             COMUNE|COMUNE_A|CC_UTS|        Shape_Leng|        Shape_Area|
+--------------------+-------+-------+--------+------+-------+-------+---------+-------------------+--------+------+------------------+------------------+
|POLYGON ((404703....|      1|      1|       1|     0|    201|   1001|   001001|             AgliÃ¨|       0|     0|1.80352540007e+004|1.31462576661e+007|
|POLYGON ((380700....|      1|      1|       1|     0|    201|   1002|   001002|            Airasca|       0|     0|1.84089069884e+004|1.57393133213e+007|
|POLYGON ((364710....|      1|      1|       1|     0|    201|   1003|   001003|       Ala di Stura|       0|     0|3.18341560810e+004|4.63315602074e+007|
|POLYGON ((415942....|      1|      1|       1|     0|    201|   1004|

In [2]:
print(f"Partitions in joined_df: {joined_df.rdd.getNumPartitions()}")

Partitions in joined_df: 25


24/12/05 15:47:35 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


In [1]:
""" DF JOIN """

from sedona.spark import *
from sedona.spark import SedonaContext, Adapter
from contextlib import contextmanager
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import desc
import time
from sedona.core.formatMapper.shapefileParser import ShapefileReader

config = SedonaContext.builder().appName('sedona-example-python')\
    .config('spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider','org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')\
    .getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
print(f'master: {sedona.conf.get("spark.master")}')
print(f"def par: {sc.defaultParallelism}")

""""""
num_partitions = sc.defaultParallelism * 2
print(f"using partitions: {num_partitions}")

print()
print("DF JOIN")
print()

total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")

cens_shp_path = "/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/"
com_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/"

with get_time("Reading cens shp file"):
    cens_rdd = ShapefileReader.readToGeometryRDD(sc, cens_shp_path)

with get_time("Reading com shp file"):
    com_rdd = ShapefileReader.readToGeometryRDD(sc, com_shp_path)
    
cens_df = Adapter.toDf(cens_rdd, sedona)
com_df = Adapter.toDf(com_rdd, sedona)
com_df_renamed = com_df.selectExpr(*[f"{c} AS {c}_com" for c in com_df.columns])

joined_df = (
    com_df_renamed.select("COMUNE_com", "geometry_com")
    .alias("com")
    .join(
        cens_df.alias("cens"),
        expr("ST_Contains(com.geometry_com, cens.geometry)"),
        "inner"
    )
)

with get_time("toDf + rename cols + Spatial Join"):
    joined_df.write.format('geoparquet').mode('overwrite').save('./res_geoparquet')
    # print(f"Total features after join: {joined_df.count()}")

if((total_time)/60 < 1):
    print(f"Job done in {total_time:.2f} sec")
else:
    print(f"Job done in {(total_time)/60:.2f} min")
print()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 16:12:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


master: local[*]
def par: 8
using partitions: 16

DF JOIN

Reading cens shp file... DONE in 0.48 sec
Reading com shp file... DONE in 0.24 sec


24/12/05 16:12:16 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


toDf + rename cols + Spatial Join... DONE in 52.30 sec
Job done in 53.02 sec



In [1]:
""" SQL JOIN """

from sedona.spark import *
from sedona.spark import SedonaContext, Adapter
from contextlib import contextmanager
from pyspark.sql.functions import desc
import time
from sedona.core.formatMapper.shapefileParser import ShapefileReader

config = SedonaContext.builder().appName('sedona-example-python')\
    .config('spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider','org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider')\
    .getOrCreate()

sedona = SedonaContext.create(config)
sc = sedona.sparkContext
print(f'master: {sedona.conf.get("spark.master")}')
print(f"def par: {sc.defaultParallelism}")

""""""
num_partitions = sc.defaultParallelism * 2
print(f"using partitions: {num_partitions}")

print()
print("SQL JOIN")
print()

total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")

cens_shp_path = "/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/"
com_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/"

with get_time("Reading cens shp file"):
    cens_rdd = ShapefileReader.readToGeometryRDD(sc, cens_shp_path)

with get_time("Reading com shp file"):
    com_rdd = ShapefileReader.readToGeometryRDD(sc, com_shp_path)
    
cens_df = Adapter.toDf(cens_rdd, sedona).repartition(num_partitions)
com_df = Adapter.toDf(com_rdd, sedona)
com_df = com_df.selectExpr(*[f"{c} AS {c}_com" for c in com_df.columns]).repartition(num_partitions)

cens_df.createOrReplaceTempView("cens_view")
com_df.createOrReplaceTempView("com_view")

result = sedona.sql('''
    SELECT cens.*, 
           com.*
    FROM com_view com, cens_view cens
    WHERE ST_Contains(com.geometry_com, cens.geometry)
''')

with get_time("toDf + rename cols + SQL views + Spatial Join"):
    result.write.format('geojson').mode('overwrite').save('./res_geojson')
    # print(f"Total features after join: {result.count()}")

if((total_time)/60 < 1):
    print(f"Job done in {total_time:.2f} sec")
else:
    print(f"Job done in {(total_time)/60:.2f} min")
print()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 14:47:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


master: local[*]
def par: 8
using partitions: 16

SQL JOIN

Reading cens shp file... DONE in 0.37 sec
Reading com shp file... DONE in 0.19 sec


24/12/05 14:48:01 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


Total features after join: 99886
toDf + rename cols + SQL views + Spatial Join... DONE in 13.98 sec
Job done in 14.54 sec



In [ ]:
map = SedonaKepler.create_map(df=com_df, name="Comune")

In [ ]:
## DataFrame API spatial join, export to .shp

import time
from contextlib import contextmanager
from pyspark.sql.functions import col, expr
from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.spark import SedonaContext, Adapter
import geopandas as gpd
from shapely.wkt import loads

total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")

print("DataFrame API spatial join, export to .csv")
print()

com_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/"   
cens_shp_path = "/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/"

# Load into RDD: Eager operation 
with get_time("Reading comune shp to SpatialRDD"):
    com_rdd = ShapefileReader.readToGeometryRDD(sc, com_shp_path)
with get_time("Reading census shp to SpatialRDD"):
    cens_rdd = ShapefileReader.readToGeometryRDD(sc, cens_shp_path)


# com_rdd.rawSpatialRDD = com_rdd.rawSpatialRDD.repartition(4)
# cens_rdd.rawSpatialRDD = cens_rdd.rawSpatialRDD.repartition(4)

# RDD -> DataFrame: Lazy operation
com_df = Adapter.toDf(com_rdd, sedona)
cens_df = Adapter.toDf(cens_rdd, sedona)

# Renaming columns: Lazy operation
com_df_renamed = com_df.selectExpr(*[f"{c} AS {c}_com" for c in com_df.columns])
cens_df_renamed = cens_df.selectExpr(*[f"{c} AS {c}_cens" for c in cens_df.columns])

# com_df_renamed = com_df.selectExpr(*[f"{c} AS {c}_com" for c in com_df.columns]).repartition(8)
# cens_df_renamed = cens_df.selectExpr(*[f"{c} AS {c}_cens" for c in cens_df.columns]).repartition(8)

# SPATIAL JOIN
# join, drop, withColumn, cache(): Lazy operatoons
joined_df = (
    com_df_renamed.select("COMUNE_com", "geometry_com")
    .alias("com")
    .join(
        cens_df_renamed.alias("cens"),
        expr("ST_Contains(com.geometry_com, cens.geometry_cens)"),
        "inner"
    )
)

joined_df = joined_df.drop("geometry_com")
joined_df = joined_df.withColumn("geometry", expr("ST_AsText(geometry_cens)")).drop("geometry_cens")
# joined_df = joined_df.repartition(8)
# joined_df.cache()

# CHCKPT 1: `.write()` triggers execution of all "Lazy operations" above
with get_time("Executing join and write to .shp"):
    pandas_df = joined_df.toPandas()
    pandas_df["geometry"] = pandas_df["geometry"].apply(loads)
    gdf = gpd.GeoDataFrame(pandas_df, geometry="geometry")

    # gdf.set_crs(epsg=4326, inplace=True)

    gdf.to_file("./joined_shp", driver="ESRI Shapefile")
    
if((total_time)/60 < 1):
    print(f"Job done in {total_time:.2f} sec")
else:
    print(f"Job done in {(total_time)/60:.2f} min")
    
print(f"Total features after join: {joined_df.count()}")


In [ ]:
com_rdd.rawSpatialRDD.getNumPartitions()

In [ ]:
# ## GeoPandas DF join
# ## ISSUE: EXHAUSTS MEMORY
 
# import geopandas as gpd
# import time
# from contextlib import contextmanager
# from pyspark.sql.functions import col, expr

# total_time = 0

# @contextmanager
# def get_time(task_name):
#     start = time.time()
#     yield
#     elapsed = time.time() - start
#     global total_time
#     total_time += elapsed
#     print(f"{task_name}... DONE in {(elapsed/60):.2f} min" if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")

# print("DF API join using GeoPandas")
# print()

# # Load into RDD: Eager operation
# with get_time("Reading comune shp to GeoPandas DF"):
#     com_gdf = gpd.read_file("/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/Com01012024_WGS84.shp")

# with get_time("Reading census shp to GeoPandas DF"):
#     cens_gdf = gpd.read_file("/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/R03_21_WGS84.shp")

# # GeoPandas DataFrame -> Sedona DataFrame: Lazy operation
# com_sdf = sedona.createDataFrame(com_gdf)
# cens_sdf = sedona.createDataFrame(cens_gdf)

# # Renaming columns: Lazy operation
# com_df_renamed = com_sdf.selectExpr(*[f"{c} AS {c}_com" for c in com_sdf.columns])  # Lazy
# census_df_renamed = cens_sdf.selectExpr(*[f"{c} AS {c}_cens" for c in cens_sdf.columns])  # Lazy

# # Spatial join
# # join, drop, withColumn, cache():  Lazy operation
# joined_df = com_df_renamed.alias("com").join(
#     census_df_renamed.alias("cens"),
#     expr("ST_Intersects(com.geometry_com, cens.geometry_cens)"),
#     "inner"
# )
# joined_df = joined_df.drop("geometry_com")
# joined_df = joined_df.withColumn("geometry", expr("ST_AsText(geometry_cens)")).drop("geometry_cens")
# joined_df.cache()

# # CHCKPT 1: `.write()` triggers execution of all "Lazy operations" above
# with get_time("Executing join and write to .csv"):
#     joined_df.write.mode("overwrite").option("header", "true").csv("./gdf_joined_csv")

# print(f"Total features aftre join: {joined_df.count()}")

# print(f"Total time: {(total_time/60):.2f} min" if total_time >= 60 else f"Total time: {total_time:.2f} sec")


In [ ]:
# SpatialRDD JOIN result -> DF -> .csv

from sedona.core.formatMapper.shapefileParser import ShapefileReader
from sedona.core.enums.grid_type import GridType
from sedona.core.enums.index_type import IndexType  GridType.KDBTREE
from sedona.spark import SedonaContext, Adapter
from sedona.core.spatialOperator import JoinQuery
from pyspark.sql.functions import col, expr
from contextlib import contextmanager
from pyspark.sql.functions import col

total_time = 0

@contextmanager
def get_time(task_name):
    start = time.time()
    yield
    elapsed = time.time() - start
    global total_time
    total_time += elapsed
    print(f"{task_name}... DONE in {(elapsed/60):.2f} min" \
          if elapsed >= 60 else f"{task_name}... DONE in {elapsed:.2f} sec")

print("SpatialRDD JOIN result -> DF -> .csv")
print()

com_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/"   
cens_shp_path = "/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/"

# Load into RDD: Eager operation 
with get_time("Reading comune shp to SpatialRDD"):
    com_rdd = ShapefileReader.readToGeometryRDD(spark.sparkContext, com_shp_path)
with get_time("Reading census shp to SpatialRDD"):
    cens_rdd = ShapefileReader.readToGeometryRDD(spark.sparkContext, cens_shp_path)
    
# .analyze(): calculates metadata like bounds, counts
with get_time("com_rdd.analyze()"):
    com_rdd.analyze()
with get_time("cens_rdd.analyze()"):
    cens_rdd.analyze()

# Spatial Partitioning + Indexing: LAzy operations
com_rdd.spatialPartitioning(GridType.KDBTREE)
cens_rdd.spatialPartitioning(com_rdd.getPartitioner())
com_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)
cens_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)

# RDD Join + toDf: Lazy operations
join_result = JoinQuery.SpatialJoinQueryFlat(
    com_rdd, cens_rdd, useIndex=True, considerBoundaryIntersection=False
)
join_rdd_df = Adapter.toDf(join_result, com_rdd.fieldNames, cens_rdd.fieldNames, spark)

def unique_cols(df):
    seen = {}
    new_cols = []

    for col_name in df.columns:
        lower_col = col_name.lower()
        if lower_col in seen:
            seen[lower_col] += 1
            new_col = f"{col_name}_dup_{seen[lower_col] - 1}"
        else:
            seen[lower_col] = 1
            new_col = col_name
        new_cols.append(new_col)

    df = df.toDF(*new_cols)
    return df

join_rdd_df = unique_cols(join_rdd_df)

join_rdd_df = join_rdd_df.drop("geom_1")
# join_rdd_df = join_rdd_df.withColumn("geom2_", expr("ST_AsText(geom_2)")).drop("geom_2")
# # join_rdd_df.cache()

# # CHCKPT 1: `.write()` triggers execution of all "Lazy operations" above
# with get_time("Executing join and write to .csv"):
#     join_rdd_df.write.mode("overwrite").option("header", "true").csv("./RDD_df_joined_csv")

# if((total_time)/60 < 1):
#     print(f"Job done in {total_time:.2f} sec")
# else:
#     print(f"Job done in {(total_time)/60:.2f} min")
    
# print(f"Total features after join: {join_rdd_df.count()}")


In [ ]:
join_rdd_df = join_rdd_df.drop("COD_ZIC")
join_rdd_df.write.mode("overwrite").option("header", "true").csv("./RDD_df_joined_csv")

In [ ]:
join_rdd_df.columns

In [ ]:
test_join_rdd_df.columns

In [ ]:
# join_rdd_df.rdd.getNumPartitions()
join_rdd_df.explain()


In [ ]:
print(f"Total time: {(total_time/60):.2f} min" if total_time >= 60 else f"{total_time:.2f} sec")

In [ ]:
# SpatialRDD JOIN + result to DF

import time
from sedona.core.spatialOperator import JoinQuery

join_result = JoinQuery.SpatialJoinQueryFlat(
    com_rdd, census_rdd, useIndex=True, considerBoundaryIntersection=True
)

start = time.time()
join_rdd_df = Adapter.toDf(join_result, com_rdd.fieldNames, census_rdd.fieldNames, spark)
end = time.time()
print(f"toDf(): {(end-start):.2f} sec")

start = time.time()
print(join_rdd_df.count())
end = time.time()

print(f"count(): {(end-start)/60:.2f} min")

In [ ]:
## ShapeFile to SpatialRDD

from sedona.core.formatMapper.shapefileParser import ShapefileReader

# reg_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Reg01012024/"
# prov_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/ProvCM01012024/"
com_shp_path = "/opt/workspace/my_data/Dati Use Case/Limiti01012024/Com01012024/"
census_shp_path = "/opt/workspace/my_data/Dati Use Case/Sezioni_censimento_Lombardia_R03_21/SHP/"


# reg_rdd = ShapefileReader.readToGeometryRDD(spark.sparkContext, reg_shp_path)
# prov_rdd = ShapefileReader.readToGeometryRDD(spark.sparkContext, prov_shp_path)
com_rdd = ShapefileReader.readToGeometryRDD(spark.sparkContext, com_shp_path)
census_rdd = ShapefileReader.readToGeometryRDD(spark.sparkContext, census_shp_path)

In [ ]:
## SpatialRDD to DF

from sedona.spark import SedonaContext, Adapter

# reg_df = Adapter.toDf(reg_rdd, spark)
# prov_df = Adapter.toDf(prov_rdd, spark)
com_df = Adapter.toDf(com_rdd, spark)
census_df = Adapter.toDf(census_rdd, spark)

In [ ]:
# SpatialRDD JOIN + result to DF

from sedona.core.spatialOperator import JoinQuery

join_result = JoinQuery.SpatialJoinQueryFlat(
    com_rdd, census_rdd, useIndex=True, considerBoundaryIntersection=True
)

join_rdd_df = Adapter.toDf(join_result, com_rdd.fieldNames, census_rdd.fieldNames, spark)

start = time.time()
print(join_rdd_df.count())
end = time.time()

print(f"{(end-start)/60:.2f} min")

In [ ]:
## Spatial Partitiotning and  Indeing + SpatialRDD API operations

from sedona.core.enums.grid_type import GridType
from sedona.core.enums.index_type import IndexType
from sedona.spark import SedonaContext, Adapter

# reg_rdd.analyze()
# prov_rdd.analyze()
com_rdd.analyze()
census_rdd.analyze()

# reg_rdd.spatialPartitioning(GridType.KDBTREE)
# prov_rdd.spatialPartitioning(reg_rdd.getPartitioner())
com_rdd.spatialPartitioning(GridType.KDBTREE)
census_rdd.spatialPartitioning(com_rdd.getPartitioner())

# reg_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)
# prov_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)
com_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)
census_rdd.buildIndex(IndexType.RTREE, buildIndexOnSpatialPartitionedRDD=True)

In [ ]:
# SpatialRDD JOIN + result to DF

from sedona.core.enums.grid_type import GridType
from sedona.core.enums.index_type import IndexType
from sedona.core.spatialOperator import JoinQuery

join_result = JoinQuery.SpatialJoinQueryFlat(
    com_rdd, census_rdd, useIndex=True, considerBoundaryIntersection=True
)

join_rdd_df = Adapter.toDf(join_result, com_rdd.fieldNames, census_rdd.fieldNames, spark)

start = time.time()
print(join_rdd_df.count())
end = time.time()

print(f"{(end-start)/60:.2f} min")

In [ ]:
## DataFrame API spatial join, export to .csv

import time
from pyspark.sql.functions import col, expr

com_df_renamed = com_df.selectExpr(*[f"{c} AS {c}_com" for c in com_df.columns])
census_df_renamed = census_df.selectExpr(*[f"{c} AS {c}_cens" for c in census_df.columns])
test = com_df_renamed.alias("com").join(census_df_renamed.alias("cens"), expr("ST_Intersects(com.geometry_com, cens.geometry_cens)"), "inner")
test = test.drop("geometry_com")
test = test.withColumn("geometry", expr("ST_AsText(geometry_cens)")).drop("geometry_cens")
# test = test.withColumn("geometry", expr("ST_SetSRID(ST_GeomFromText(geometry), 4326)"))

start = time.time()
# test.write.mode("overwrite").csv("./df_joined_csv")
end = time.time()

if((end-start)/60 < 1):
    print(f"{end-start:.2f} sec")
else:
    print(f"{(end-start)/60:.2f} min")

In [ ]:
## SpatialRDD JOIN + result to DF

from sedona.core.enums.grid_type import GridType
from sedona.core.enums.index_type import IndexType
from sedona.core.spatialOperator import JoinQuery
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.sql.functions as F

join_result = JoinQuery.SpatialJoinQueryFlat(
    reg_rdd, census_rdd, useIndex=True, considerBoundaryIntersection=True
)

join_rdd_df = Adapter.toDf(join_result, reg_rdd.fieldNames, census_rdd.fieldNames, spark)

start = time.time()
print(join_rdd_df.count())
end = time.time()

print(f"{(start-end)/60:.2f} min")

In [ ]:
# test_join_df = reg_df.alias("reg").join(prov_df.alias("prov"), expr("reg.COD_REG == prov.COD_REG"), "inner")

polygon_df = joined_df.alias("reg").filter(expr("ST_GeometryType(reg.geometry) = 'Polygon'"))

polygon_df.show()